In [1]:
import os
import pandas as pd
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
models = ["tinycnn", "simplecnn", "resnet18", "mobilenetv2"]
splits = ["Label_skew", "Concept_shift", "Feature_skew", "Pathological", "Quantity_skew", "IID_equal",
          "Dirichlet_label"]
clients = [0, 1, 2]
log_dir = "/content/drive/MyDrive/Colab Notebooks/logs/contribution_logs"

In [3]:
contrib_summary = []

for model in models:
    for split in splits:
        # Load global accuracy with all clients
        all_file = f"{log_dir}/log_global_metrics_{model}_with_all_{split}.csv"
        acc_with_all = pd.read_csv(all_file)["global_accuracy"].mean()

        for cid in clients:
            excl_file = f"{log_dir}/log_global_metrics_{model}_excl_client_{cid}_{split}.csv"
            acc_without_client = pd.read_csv(excl_file)["global_accuracy"].mean()

            contribution = acc_with_all - acc_without_client
            contrib_summary.append({
                "Model": model,
                "Split": split,
                "Client": cid,
                "Acc_with_all": acc_with_all,
                "Acc_without_client": acc_without_client,
                "Contribution": contribution
            })

# Save to CSV
df_contrib = pd.DataFrame(contrib_summary)
df_contrib.to_csv("/content/drive/MyDrive/Colab Notebooks/client_contribution_summary.csv", index=False)